In [29]:
import cv2
import matplotlib.pyplot as plt
import skimage.segmentation as seg
import skimage.filters as filt
import skimage.morphology as morph
import skimage.draw as draw
from scipy import ndimage
from mpl_toolkits.mplot3d import Axes3D
from scipy.integrate import cumtrapz
import scipy.ndimage.filters as filters
import os
import numpy as np
import pandas as pd
from plantcv import plantcv as pcv 

In [30]:
class options:
    def __init__(self):
        self.debug = "none"
        self.writeimg= 'False' 
        self.result = "features_metadata.json"
        self.outdir = ""
# Get options
args = options()

# Set debug to the global parameter 
pcv.params.debug = args.debug
pcv.params.debug = args.debug

In [88]:
def curvature(x, y, xc, yc, r):
    # Shift coordinates to make the center of the circle the origin
    x_shifted = x - xc
    y_shifted = y - yc
    # Calculate distance from each point to the center of the circle
    d = np.sqrt(x_shifted**2 + y_shifted**2)
    # Calculate the curvature only for points inside the circle
    inside_circle = d <= r
    dx_dt = np.gradient(x[inside_circle])
    dy_dt = np.gradient(y[inside_circle])
    d2x_dt2 = np.gradient(dx_dt)
    d2y_dt2 = np.gradient(dy_dt)
    curvature = (dx_dt * d2y_dt2 - d2x_dt2 * dy_dt) / (dx_dt**2 + dy_dt**2)**(3/2)
    # Pad the curvature array with zeros for the points outside the circle
    curvature_padded = np.zeros_like(d)
    curvature_padded[inside_circle] = curvature
    return curvature_padded


def integral_curvature(x, y, xc, yc, r):
    k = curvature(x, y, xc, yc, r)
    return cumtrapz(k, initial=0)


def get_integral_curvature(i,x,y, r):
    # Calculate the integral curvature for a circle centered at (x[i], y[i]) with radius r
    int_curv = integral_curvature(x, y, x[i], y[i], r)[-1]
    return int_curv


def get_area_measure(contour,i,radius, thresh):
    # Approximate the contour with a circle centered at the current point
    circle_mask = np.zeros_like(gray)
    cv2.circle(circle_mask, (contour[i][0][0], contour[i][0][1]), radius, (255, 255, 255), -1)
    intersection_mask = cv2.bitwise_and(circle_mask, thresh)
    intersection_area = np.sum(intersection_mask) / 255
    circle_area = np.pi * radius ** 2
    intersection_fraction = intersection_area / circle_area
    # Compute the curvature value for the current point
    return intersection_fraction


def get_curvature_arc_length(contour,i,x,y,radius,circle_perimeters):
    curvature_arr = []
    # Approximate the contour with a circle centered at the current point
    (cx, cy) = x[i], y[i]
    # Calculate the length of the part of the circle's circumference that is inside the object
    chord_length = 2 * np.sqrt(radius**2 - ((radius**2-(x[i]-cx)**2-(y[i]-cy)**2))/4)
    arc_length = 2 * np.arcsin(chord_length / (2 * radius))
    circle_perimeters.append(arc_length)
    if len(circle_perimeters) > 0:
        # Calculate the average perimeter of the fitted circles
        circle_perimeter = np.mean(circle_perimeters)
    else:
        # Use the radius as an estimate for the perimeter
        circle_perimeter = 2 * np.pi * radius

    # Calculate perimeter of the contour
    contour_perimeter = cv2.arcLength(contour, True)
    # Calculate the fraction of the circle's perimeter contained inside the object
    fraction = circle_perimeter / contour_perimeter
    arc_len = fraction * arc_length
    return arc_len


def get_curvatures(contours, radius_arr, thresh):
    curvature_arr=[]
    
    # Loop over each contour
    for contour in contours:
        curr = []
        # Precompute the coordinates of all points in the contour
        x, y = contour[:, 0, 0], contour[:, 0, 1]
        # Compute the maximum and minimum x and y coordinates of the contour
        min_x, max_x = np.min(x), np.max(x)
        min_y, max_y = np.min(y), np.max(y)
        # Compute the thresholded image for the current contour
        # mask = np.zeros_like(gray)
        # cv2.drawContours(mask, [contour], 0, (255, 255, 255), -1)
        # thresh = cv2.bitwise_and(gray, mask)
        # Compute the area measure for each point in the contour
        for radius in radius_arr:
            circle_perimeters = []
            arc_length = 0  # Initialize arc_length to 0
            curr = []
            for i in range(len(contour)):
                curr_area = get_area_measure(contour,i,radius,thresh)
                # curr_arc = get_curvature_arc_length(contour,i,x,y,radius,circle_perimeters)
                # curr_int = get_integral_curvature(i,x,y, radius)
                curr.append(curr_area) # remove
                #curr.append(curr_arc)
                #curr.append(curr_int) # add
            curvature_arr.append(curr)
    return curvature_arr
            #print( f'area={curr_area} arc ={curr_arc} curr int = {curr_int}')

In [89]:
# # Convert RGB to HSV and extract the saturation channel
# s = pcv.rgb2gray_hsv(rgb_img=img, channel='s')
# # Threshold the saturation image
# s_thresh = pcv.threshold.binary(gray_img=s, threshold=85, max_value=255, object_type='light')


# specify path to the folder containing image files
path = "../../leafsnap/leafsnap-dataset/dataset/images/lab_/2"
thresh = []
counter = 0

# initialize empty lists for features and labels
features = []
labels = []

# loop through all subfolders in the path
for foldername in os.listdir(path):
    if counter == 2:
        break
    
    folderpath = os.path.join(path, foldername)
    if not os.path.isdir(folderpath):
        continue
    
    # loop through all image files in the subfolder
    for filename in os.listdir(folderpath):
        filepath = os.path.join(folderpath, filename)
        if not os.path.isfile(filepath):
            continue
        # try:
        contours = pcv_getContours(filepath)
        radius_arr = [10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,190,200,210,220,230,240,250]
        ini_curvature_arr = get_curvatures(contours, radius_arr, thresh)
        curvature_arr = np.array(ini_curvature_arr)
        x = np.arange(curvature_arr.shape[1]) # x-axis is the number of contour points
        y = np.arange(curvature_arr.shape[0]) # y-axis is the different scales
        X, Y = np.meshgrid(x, y)
        
        hist_values = []
        for i in range(curvature_arr.shape[0]):
            counts, bin_edges = np.histogram(curvature_arr[i], bins=21) # adjust the number of bins as needed
            hist_values.append(counts)
        data = np.array([np.array(hist) for hist in hist_values])
        X = data.flatten().tolist()
        print(X)
        features.append(X)
        labels.append(foldername)
        # except:
        #     pass
        
    print(f"Finished plant: {foldername}")
    counter += 1
    

# convert features and labels to numpy arrays
features = features
labels = np.array(labels)

# save features and labels to a pandas dataframe and export to CSV file
data = pd.DataFrame({"hist_values" : features})
data["plant"] = labels
data.to_csv("leafsnap_data_fin.csv", index=False)

[[[640 264]]

 [[639 265]]

 [[639 266]]

 ...

 [[643 192]]

 [[642 192]]

 [[641 192]]]


IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed

In [83]:
from plantcv import plantcv as pcv
import scipy.integrate

def pcv_getContours(path):
    # img, path, filename = pcv.readimage(filename=args.image)
    # img, path, filename = pcv.readimage(filename="../../leafsnap/leafsnap-dataset/dataset/images/lab/abies_concolor/ny1157-01-1.jpg")
    img, path, filename = pcv.readimage(filename=path)

    # Resize image using resize function, with interpolation, and default interpolation method
    # img = pcv.transform.resize(img=img, size=(400, 300), interpolation="auto")

    # Convert RGB to HSV and extract the saturation channel
    s = pcv.rgb2gray_hsv(rgb_img=img, channel='s')
    # Threshold the saturation image
    s_thresh = pcv.threshold.binary(gray_img=s, threshold=85, max_value=255, object_type='light')
    # Median Blur
    s_mblur = pcv.median_blur(gray_img=s_thresh, ksize=5)
    s_cnt = pcv.median_blur(gray_img=s_thresh, ksize=5)
    # Convert RGB to LAB and extract the Blue channel
    #b = pcv.rgb2gray_lab(gray_img=img, channel='b')
    b = pcv.rgb2gray_lab(rgb_img=img, channel='b')
    # Threshold the blue image
    b_thresh = pcv.threshold.binary(gray_img=b, threshold=160, max_value=255, object_type='light')
    b_cnt = pcv.threshold.binary(gray_img=b, threshold=160, max_value=255, object_type='light')
    # Fill small objects
    # b_fill = pcv.fill(b_thresh, 10)
    # Join the thresholded saturation and blue-yellow images
    bs = pcv.logical_or(bin_img1=s_mblur, bin_img2=b_cnt)
    # Apply Mask (for VIS images, mask_color=white)
    #masked = pcv.apply_mask(rgb_img=img, mask=bs, mask_color='white')
    masked = pcv.apply_mask(img=img, mask=bs, mask_color='white')
    # Convert RGB to LAB and extract the Green-Magenta and Blue-Yellow channels
    masked_a = pcv.rgb2gray_lab(rgb_img=masked, channel='a')
    masked_b = pcv.rgb2gray_lab(rgb_img=masked, channel='b')
    # Threshold the green-magenta and blue images
    maskeda_thresh = pcv.threshold.binary(gray_img=masked_a, threshold=115, max_value=255, object_type='dark')
    maskeda_thresh1 = pcv.threshold.binary(gray_img=masked_a, threshold=135, max_value=255, object_type='light')
    maskedb_thresh = pcv.threshold.binary(gray_img=masked_b, threshold=128, max_value=255, object_type='light')
    # Join the thresholded saturation and blue-yellow images (OR)
    ab1 = pcv.logical_or(bin_img1=maskeda_thresh, bin_img2=maskedb_thresh)
    ab = pcv.logical_or(bin_img1=maskeda_thresh1, bin_img2=ab1)
    # Fill small objects
    ab_fill = pcv.fill(bin_img=ab, size=200)
    # Apply mask (for VIS images, mask_color=white)
    #masked2 = pcv.apply_mask(rgb_img=masked, mask=ab_fill, mask_color='white')
    masked2 = pcv.apply_mask(img=masked, mask=ab_fill, mask_color='white')
    skeleton = pcv.morphology.skeletonize(mask=masked2)
    # Identify objects
    id_objects, obj_hierarchy = pcv.find_objects(img=img, mask=ab_fill)
    dimensions = img.shape
    # height, width, number of channels in image
    height = img.shape[0] - (img.shape[0] * (30 / 100))
    width = img.shape[1] - (img.shape[1] * (30 / 100))
    # Define ROI
    roi1, roi_hierarchy= pcv.roi.rectangle(img=masked2, x=100, y=100, h=height, w=width)
    # # Auto-define the ROI using the "auto_roi" function
    # roi1, roi_heirarchy = pcv.auto_roi(masked2)
    # Decide which objects to keep
    roi_objects, hierarchy3, kept_mask, obj_area = pcv.roi_objects(img=img, roi_contour=roi1, 
                                                                   roi_hierarchy=roi_hierarchy, 
                                                                   object_contour=id_objects, 
                                                                   obj_hierarchy=obj_hierarchy,
                                                                   roi_type='partial')
    # Object combine kept objects
    global thresh
    obj, thresh = pcv.object_composition(img=img, contours=roi_objects, hierarchy=hierarchy3)
    return obj